In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install igraph

     |████████████████████████████████| 3.2 MB 5.3 MB/s 


In [3]:
pip install stellargraph

     |████████████████████████████████| 435 kB 5.3 MB/s 


In [4]:
from joblib import Parallel, delayed
from datetime import time, datetime
from tqdm import tqdm

import time
from datetime import datetime

from igraph import *
import networkx as nx
from matplotlib import pyplot as plt
import pandas as pd

from stellargraph import StellarGraph

import numpy as np

import math

In [5]:
def humanize_unixtime(unix_time):
    time = datetime.fromtimestamp(int(unix_time)).strftime('%d-%m-%Y %H.%M.%S')
    return time

In [6]:
def get_24hr_details(input_type, month,unix_time):
    
    #hashes list to store all the hashes
    details_dict = dict()
    
    #read files
    if input_type == "inputs":
        filename = "/content/drive/MyDrive/edges2010/updated/inputs2010_{}.txt".format(str(month+1))
    if input_type == "outputs":
        filename = "/content/drive/MyDrive/edges2010/updated/outputs2010_{}.txt".format(str(month+1))

    with open(filename) as file:
        for line in file:
            #split by tab char to get the details present
            line = line.replace('\n', '')
            print("line",line)
            details = line.split("\t")

            #hash present at the second value
            #append in the list
            #details_dict[details[1]] = dict()
            unix_time_24 = unix_time + 24 * 60 * 60
            print("unix_time",unix_time)
            print("unix_time_24",unix_time_24)

            if details[0] >= str(unix_time) and details[0] <= str(unix_time_24):
                details_dict[details[1]] = dict()
                if input_type == "inputs":
                    #details_dict[details[1]]["input"] = details[2:]
                    details_dict[details[1]]['time'] = details[0]
                    details_dict[details[1]]["input"] = details[2:]
                    details_dict[details[1]]["human"] = humanize_unixtime(details[0])
                
                if input_type == "outputs":
                    #details_dict[details[1]]["output"] = details[2:]
                    details_dict[details[1]]["output"] = details[2:]
                    details_dict[details[1]]["time"] = details[0]
                    details_dict[details[1]]["human"] = humanize_unixtime(details[0])
            
    return details_dict

In [7]:
def get_addr_node_details(g,addr_list):
    
    #defining lists to be added to df
    total_neighbours_count_list = []
    successor_neighbours_count_list = []
    predecessor_neighbours_count_list = []
    Income_list = []
    Expenditure_list = []
    Balance_list = []

    for i in range(len(addr_list)):
        total_neighbours_count_list.append(g.degree[addr_list[i]])
        successor_neighbours_count_list.append(len(list(g.successors(addr_list[i]))))
        predecessor_neighbours_count_list.append(len(list(g.predecessors(addr_list[i]))))
        #print("neighbours done for:",addr_list[i])


        #successsors nodes and weights
        my_node_successors_list = []
        my_node_successors_list= list(g.successors(addr_list[i]))
        sum_of_successor_amount = 0

        for j in range (0,len(my_node_successors_list)):
            #print("successor for loop entered for:",addr_list[i])
            my_node_successors = my_node_successors_list[j]
            #print("checking for succssor node:",my_node_successors," of ",addr_list[i])
            my_node_weight = g[addr_list[i]][my_node_successors]['weight'] #getting weight of directed edge from my_node to successors
            #print("checked for weight: ",my_node_weight, " of succssor node: ",my_node_successors," of ",addr_list[i])
            sum_of_successor_amount += my_node_weight #adding up weights
            #print("added weights",sum_of_successor_amount, "for",addr_list[i])
        #print("successor done for:",addr_list[i])
    
        #predecessors nodes and weights
        my_node_predecessors_list = []
        my_node_predecessors_list= list(g.predecessors(addr_list[i]))
        sum_of_predecessor_amount = 0

        for j in range (0,len(my_node_predecessors_list)):
            my_node_predecessors = my_node_predecessors_list[j]
            my_node_weight = g[my_node_predecessors][addr_list[i]]['weight'] #getting weight of directed edge from predecessors to my_node
            sum_of_predecessor_amount += my_node_weight
        #print("predecessor done for:",addr_list[i])

        Income_list.append(sum_of_predecessor_amount)
        Expenditure_list.append(sum_of_successor_amount)
        Balance_list.append(sum_of_predecessor_amount-sum_of_successor_amount)    
        #print("amounts done:",addr_list[i])

    
    
    #subgraph for my_node
    #nodelist = [my_node]
    #nodelist.extend(my_node_successors_list)
    #nodelist.extend(my_node_predecessors_list)
    #g2 = g.subgraph(nodelist)
    #pos=nx.circular_layout(g2)
    ##pos = nx.spring_layout(g2, seed=7)
    #nx.draw(g2, pos, with_labels=True)

    return total_neighbours_count_list,successor_neighbours_count_list,predecessor_neighbours_count_list,Income_list,Expenditure_list,Balance_list

In [8]:
def create_graph(g,updated_dict):
  
      for key, value in tqdm(updated_dict.items()):
        
        #print("Current transaction hash {}".format(key))
        hash_of_transc = key
        
        
        input_details_of_hash_of_transc = value["input"]
        output_details_of_hash_of_transc = value["output"]
        
        #print("Transaction input details {}".format(input_details_of_hash_of_transc))
        #print("Transaction output details {}".format(output_details_of_hash_of_transc))
        
        #going over all input transactions
        for i in range(0, len(input_details_of_hash_of_transc), 2):
            
            input_hash = input_details_of_hash_of_transc[i]
            output_index = input_details_of_hash_of_transc[i+1]
            
            #print(input_hash)
            #print(output_index)
            if input_hash not in updated_dict.keys():
                #g.add_vertex(input_hash)
                #print("no address")
                pass
            else:
                #g.add_vertex(input_hash)
                #get output details of that input transaction
                get_amounts = updated_dict[input_hash]["output"]
                
                #print(get_amounts)
                address = get_amounts[int(output_index)]
                amount = get_amounts[2*int(output_index)+1]
                
                
                #add input transaction node
                g.add_node(input_hash, s='s', label='transaction', color="r")
                
                #add address node
                g.add_node(address, s ='s', label='address', color="b")
                
                #add output transaction node
                g.add_node(hash_of_transc, s='o', label='transaction', color="r")
                
                #add edge between input transaction and address
                g.add_edge(input_hash, address, weight=int(amount))
                
                #add edge between address and output transaction
                g.add_edge(address, key, weight=int(amount))

      return g

In [9]:
#intitating graph
g = nx.DiGraph()

#considering data files till this month
month = 6

#input dict to store all the input details
input_dict = dict()

#output dict to store all the output details
output_dict = dict()


#dt = datetime(2010,1,23,20,17,11)
day1 = datetime(2010,1,1)
unixtimestamp_start = int((day1 - datetime(1970,1,1)).total_seconds())

if month==12:
    day_end = datetime(2011,1,1)
else:
    day_end = datetime(2010,month+1,1)

unixtimestamp_end = int((day_end - datetime(1970,1,1)).total_seconds())




iteration_count = math.ceil((unixtimestamp_end - unixtimestamp_start)/(24 * 60 * 60))
print("iteration_count = ",iteration_count)

df = [None] * iteration_count
j = 0
for unixtimestamp in range(unixtimestamp_start,unixtimestamp_end,24 * 60 * 60):
          #calling function for input
          start_time = time.time()
          inp = Parallel(n_jobs=-1)(delayed(get_24hr_details)("inputs", i,unixtimestamp) for i in tqdm(range(month)))
          for d in inp:
              input_dict.update(d)

          end_time = time.time()
          print("Time taken to read the input file data is {}".format(end_time - start_time))

          #print input_dict
          print("Total details in input dict is: {}".format(len(input_dict)))

          #calling function for output
          start_time = time.time()
          inp = Parallel(n_jobs=-1)(delayed(get_24hr_details)("outputs", o,unixtimestamp) for o in tqdm(range(month)))
          for d in inp:
              output_dict.update(d)

          end_time = time.time()
          print("Time taken to read the output file data is {}".format(end_time - start_time))

          #print output_dict
          print("Total details in output dict is: {}".format(len(output_dict)))

          #merge input and output data
          for key, value in output_dict.items():
              input_details = input_dict[key]
              output_dict[key]["input"] = input_details["input"]

          print("Total details in linked dict is: {}".format(len(output_dict)))

          updated_dict = dict(sorted(output_dict.items(), key=lambda item: item[1]['time']))


          #create the graph
          g = nx.DiGraph()
          g = create_graph(g,updated_dict)
          
          #plot the graph for each g
          square_foo_bar = StellarGraph.from_networkx(g)
          print(square_foo_bar.info())


          #pick only the address nodes from each g
          all_node_list = []
          addr_list = []
          all_node_list = list(g.nodes)
          i=0
          for i in range(len(all_node_list)):
              if g.nodes[all_node_list[i]]['label'] == "address":
                  addr_list.append(all_node_list[i])
          
          print("len of addr_list:",len(addr_list))

          #call address node function for features
          total_neighbours_count_list,successor_neighbours_count_list,predecessor_neighbours_count_list,Income_list,Expenditure_list,Balance_list = get_addr_node_details(g,addr_list)


          #converting address features into dataframe
          df[j] = pd.DataFrame(np.column_stack([addr_list, predecessor_neighbours_count_list, successor_neighbours_count_list,total_neighbours_count_list,Income_list,Expenditure_list,Balance_list]), 
                                columns=['Address','Predecessor_Neighbours', 'Successor_Neighbours','Total_neighbours','Income','Expenditure','Balance'])
          j+=1

#append all the df
print ("lendght of df = ",len(df))
for i in range(0,len(df)):
    if i ==0:
        result_df = df[i].copy()
    else:
        result_df = result_df.append(df[i])

#convert df into csv file
result_df.to_csv('/content/drive/MyDrive/Blockchain Output CSVs/Jan_to_Jun_2010_24hrDivision.csv')

iteration_count =  181


100%|██████████| 6/6 [00:01<00:00,  5.70it/s]


Time taken to read the input file data is 2.698425531387329
Total details in input dict is: 2


100%|██████████| 6/6 [00:00<00:00, 37.14it/s]


Time taken to read the output file data is 1.1890888214111328
Total details in output dict is: 2
Total details in linked dict is: 2


...

Time taken to read the input file data is 0.29965853691101074
Total details in input dict is: 7654


100%|██████████| 6/6 [00:00<00:00, 206.28it/s]


Time taken to read the output file data is 0.22716164588928223
Total details in output dict is: 7654
Total details in linked dict is: 7654


100%|██████████| 7654/7654 [00:00<00:00, 35018.00it/s]


StellarDiGraph: Directed multigraph
 Nodes: 9340, Edges: 15486

 Node types:
  transaction: [5922]
    Features: none
    Edge types: transaction-default->address
  address: [3418]
    Features: none
    Edge types: address-default->transaction

 Edge types:
    transaction-default->address: [8939]
        Weights: range=[1e+06, 2.3e+12], mean=1.63234e+10, std=9.03455e+10
        Features: none
    address-default->transaction: [6547]
        Weights: range=[1e+06, 2.3e+12], mean=1.91752e+10, std=1.02936e+11
        Features: none
len of addr_list: 3418
lendght of df =  181


In [10]:
result_df.head(100)

,Address,Predecessor_Neighbours,Successor_Neighbours,Total_neighbours,Income,Expenditure,Balance
0,500000000000,1,1,2,4988000000,4988000000,0
0,500000000000,1,1,2,4988000000,4988000000,0
0,500000000000,1,1,2,4988000000,4988000000,0
1,1KqBVJGQBrT5RYWQE8oEdH6UAHLLCgp79E,1,1,2,500000000,500000000,0
0,500000000000,1,1,2,4988000000,4988000000,0
...,...,...,...,...,...,...,...
2,574900000000,1,1,2,74000000,74000000,0
3,1ET9ufvZEAMUmQqLHYbuixjuGjHv9CGmy,1,1,2,100000000,100000000,0
4,5000000,1,1,2,4575000000,4575000000,0
5,1000000,1,1,2,4574000000,4574000000,0


In [11]:
len(result_df)

191638